In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

columns = ['ID','Gender','Region_Code','Occupation','Channel_Code','Is_Active']
for c in columns: 
  if train[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)

display(train.head())
display(test.head())

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,255836,0,73,18,1,2,43,No,1045696,0,0
1,187146,0,30,27,2,0,32,No,581988,0,0
2,173604,0,56,18,3,2,26,No,1484315,1,0
3,81840,1,34,20,2,0,19,No,470454,0,0
4,299455,0,30,32,2,0,33,No,886787,0,0


,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,314009,1,29,4,1,0,25,Yes,742366,0
1,96047,1,43,18,1,1,49,NaN,925537,0
2,316206,1,31,20,2,0,14,No,215949,0
3,303077,1,29,22,1,0,33,No,868070,0
4,292640,0,29,20,1,0,19,No,657087,0


In [ ]:
train['Credit_Product'] = train['Credit_Product'].replace(['Yes','No'],[1,0])
test['Credit_Product'] = test['Credit_Product'].replace(['Yes','No'],[1,0])

In [ ]:
target = train.pop('Is_Lead')
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train,target,test_size = 0.1)

In [ ]:

params={'bagging_freq': 1, 'reg_alpha': 0.0019599943581399, 'reg_lambda': 0.028564643120722513, 'colsample_bytree': 0.24,
     'subsample': 0.8, 'learning_rate': 0.008, 'max_depth': 140, 'num_leaves': 100, 'min_child_samples': 228, 'max_bin': 238,
     'cat_smooth': 75,'metric': 'auc', 'random_state': 48,'n_estimators': 2000}
test_pred1 = np.zeros(len(test))
auc_1 = []
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=100)
for train_index, test_index in kf.split(train,target):
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target.iloc[train_index],target.iloc[test_index]
    lgbm = LGBMClassifier()
    lgbm.fit(X_train,y_train)
    preds = lgbm.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test,preds)
    auc_1.append(auc)
    test_pred1 += (lgbm.predict_proba(test)[:,1])/kf.n_splits
np.mean(auc_1)

0.8737521679109199

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
test_pred2 = np.zeros(len(test))
auc_2 = []
kf = StratifiedKFold(n_splits=50)
for train_index, test_index in kf.split(train,target):
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target.iloc[train_index],target.iloc[test_index]
    clf = LGBMClassifier()
    clf.fit(X_train,y_train)
    preds = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test,preds)
    test_pred2 += (lgbm.predict_proba(test)[:,1])/kf.n_splits
    auc_2.append(auc)
np.mean(auc_2)

0.873790990292515

In [ ]:
print(auc)

In [ ]:
test_pred = ((2*(test_pred1))+test_pred2)/3

Auc with baseline lightgbm model - 0.8791842836418373

Auc with lightgbm and simple training - 0.8781330198896204

In [ ]:
sample_submision = pd.read_csv('sample_submission.csv',index_col='ID')

In [ ]:
sample_submision.head()

In [ ]:
sample_submision['Is_Lead'] = test_pred
sample_submision.head()

,Is_Lead
ID,
VBENBARO,0.041791
CCMEWNKY,0.865215
VK3KGA9M,0.054955
TT8RPZVC,0.022972
SHQZEYTZ,0.023274


In [ ]:
sample_submision.to_csv('final_submission3.csv')